In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class Gait1DCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(Gait1DCNN, self).__init__()
        # Input shape: [batch_size, 6, 500]
        self.conv1 = nn.Conv1d(
            in_channels=6, out_channels=64, kernel_size=3, stride=1, padding=1
        )
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)

        # Calculate the size of the features after the convolution and pooling layers
        # Assuming input length is 500 and you apply pooling twice
        conv_output_size = 500 // 2 // 2
        self.fc1 = nn.Linear(128 * conv_output_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Flatten the output for the dense layer
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [39]:
model = torch.load("1D_CNN_model_nate_2.pth")
label_dict = {"hersh": 0, "nate": 1, "ryan": 2}
model.eval()  # Set the model to evaluation mode

Gait1DCNN(
  (conv1): Conv1d(6, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=12800, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [40]:
def preprocess_data(sample, crop_size, noise_level):
    # Normalize the sample
    sample_mean = sample.mean(axis=0)
    sample_std = sample.std(axis=0)
    sample = (sample - sample_mean) / sample_std

    # Random cropping
    if sample.shape[0] > crop_size:
        start = np.random.randint(0, sample.shape[0] - crop_size)
        sample = sample[start : start + crop_size, :]
    else:
        # If the sample is shorter than crop_size, you might want to pad it or handle it differently
        pass

    # Adding noise
    noise = np.random.normal(0, noise_level, sample.shape)
    sample = sample + noise

    # Reshape and convert to PyTorch tensor
    sample_tensor = torch.from_numpy(sample).float().transpose(0, 1).unsqueeze(0)
    return sample_tensor

In [41]:
# Define preprocessing parameters
crop_size = 400  # Example crop size, adjust to your needs
noise_level = 0.05  # Example noise level


def predict(model, sample):
    with torch.no_grad():  # Ensure gradients are not computed in inference mode
        preprocessed_sample = preprocess_data(sample, crop_size, noise_level)
        outputs = model(preprocessed_sample)
        _, predicted = torch.max(outputs, 1)
        return predicted.item(), outputs  # or return outputs for probabilistic interpretation


for name, label in label_dict.items():
    for i in range(1, 21):
        sample = np.loadtxt(
            f"data/{name}_{i}.txt", delimiter=","
        ) 
        prediction = predict(model, sample)
        print(f"Testing {name}_{i} | Predicted class: {prediction}")

for i in range(1, 18):
        sample = np.loadtxt(
            f"data/unrec_{i}.txt", delimiter=","
        ) 
        prediction = predict(model, sample)
        print(f"Testing unrec_{i} | Predicted class: {prediction}")

Testing hersh_1 | Predicted class: (0, tensor([[ 9.6580, -4.0114]])), Expected class 0
Testing hersh_2 | Predicted class: (0, tensor([[12.0874, -6.9814]])), Expected class 0
Testing hersh_3 | Predicted class: (0, tensor([[10.7540, -4.5375]])), Expected class 0
Testing hersh_4 | Predicted class: (0, tensor([[ 9.7383, -3.9252]])), Expected class 0
Testing hersh_5 | Predicted class: (0, tensor([[ 9.6458, -4.0185]])), Expected class 0
Testing hersh_6 | Predicted class: (0, tensor([[10.1473, -4.3212]])), Expected class 0
Testing hersh_7 | Predicted class: (0, tensor([[ 8.8072, -3.1248]])), Expected class 0
Testing hersh_8 | Predicted class: (0, tensor([[11.5090, -4.8169]])), Expected class 0
Testing hersh_9 | Predicted class: (0, tensor([[13.5040, -7.1398]])), Expected class 0
Testing hersh_10 | Predicted class: (0, tensor([[10.9668, -4.6129]])), Expected class 0
Testing hersh_11 | Predicted class: (0, tensor([[14.5363, -8.5219]])), Expected class 0
Testing hersh_12 | Predicted class: (0, t

In [42]:
for i in range(1, 11):
    sample = np.loadtxt(f"nate_data/nate_data_{i}.txt", delimiter=",")
    prediction = predict(model, sample)
    print(f"Testing nate_{i} | Predicted class: {prediction}, Expected class 1")

Testing nate_1 | Predicted class: (0, tensor([[ 7.6778, -2.4982]])), Expected class 1
Testing nate_2 | Predicted class: (0, tensor([[ 6.8828, -1.9182]])), Expected class 1
Testing nate_3 | Predicted class: (0, tensor([[ 9.9123, -4.4016]])), Expected class 1
Testing nate_4 | Predicted class: (0, tensor([[11.2481, -5.3169]])), Expected class 1
Testing nate_5 | Predicted class: (0, tensor([[11.8164, -5.7767]])), Expected class 1
Testing nate_6 | Predicted class: (0, tensor([[14.0191, -7.8180]])), Expected class 1
Testing nate_7 | Predicted class: (0, tensor([[ 8.7397, -3.8658]])), Expected class 1
Testing nate_8 | Predicted class: (0, tensor([[12.3203, -6.5569]])), Expected class 1
Testing nate_9 | Predicted class: (0, tensor([[11.6861, -5.7179]])), Expected class 1
Testing nate_10 | Predicted class: (0, tensor([[ 9.6361, -4.1645]])), Expected class 1
